In [1]:
import nltk
# tokensize http://www.nltk.org/api/nltk.tokenize.html#module-nltk.tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize, blankline_tokenize
# stem http://www.nltk.org/api/nltk.stem.html#module-nltk.stem
from nltk.stem import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
# corpus 数据集
from nltk.corpus import stopwords
# metrics评分
from nltk.metrics import edit_distance
# tag
from nltk.tag import pos_tag

In [ ]:
# 命名实体识别
# 分句-分词-词性标签-命名实体识别-实体关系识别

In [7]:
document = open("test.txt").read()

In [8]:
def test_InformationExtract(document):
    sentences = nltk.sent_tokenize(document)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
test_InformationExtract(document)

In [10]:
# 实体识别的基本技术是分块（chunking chunking chunking chunking）
# 基于正则的分块
# 名词短语分块，或 NP-分块（NP-chunking NP-chunking NP-chunking NP-chunking），寻找单独名词短语对应的块。
sentence = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"),("dog", "NN"), ("barked", "VBD"), ("at", "IN"), ("the", "DT"), ("cat", "NN")]
grammar = "NP: {<DT>?<JJ>*<NN>}" 
cp = nltk.RegexpParser(grammar) 
result = cp.parse(sentence)
print (result)

(S
  (NP the/DT little/JJ yellow/JJ dog/NN)
  barked/VBD
  at/IN
  (NP the/DT cat/NN))


In [13]:
# 要找到一个给定的句子的块结构，RegexpParser 分块器以一个没有标识符被分块的平面结构开始。
# 轮流应用分块规则，依次更新块结构。所有的规则都被调用后，返回块结构。
grammar = r"""
NP: {<DT|PP\$>?<JJ>*<NN>} # chunk determiner/possessive, adjectives and nouns
{<NNP>+} # chunk sequences of proper nouns
"""
cp = nltk.RegexpParser(grammar)
sentence = [("Rapunzel", "NNP"), ("let", "VBD"), ("down", "RP"),
("her", "PP$"), ("long", "JJ"), ("golden", "JJ"), ("hair", "NN")]
print (cp.parse(sentence))

(S
  (NP Rapunzel/NNP)
  let/VBD
  down/RP
  (NP her/PP$ long/JJ golden/JJ hair/NN))


In [16]:
# 读取分块语料库
from nltk.corpus import conll2000
print (conll2000.chunked_sents('train.txt')[99])
# NP块是名词短语 VP 块如 has already delivered；PP 块如 because of
# 如果只想要NP块
print (conll2000.chunked_sents('train.txt', chunk_types=['NP'])[99])
# 评估

(S
  (PP Over/IN)
  (NP a/DT cup/NN)
  (PP of/IN)
  (NP coffee/NN)
  ,/,
  (NP Mr./NNP Stone/NNP)
  (VP told/VBD)
  (NP his/PRP$ story/NN)
  ./.)
(S
  Over/IN
  (NP a/DT cup/NN)
  of/IN
  (NP coffee/NN)
  ,/,
  (NP Mr./NNP Stone/NNP)
  told/VBD
  (NP his/PRP$ story/NN)
  ./.)


In [18]:
# 训练基于分类器的分块器
"""
有时词性标记不足以确定一个句子应如何分块。例如：考虑下面的两个语句：

a. Joey/NN sold/VBD the/DT farmer/NN rice/NN ./.
b. Nick/NN broke/VBD my/DT computer/NN monitor/NN ./.

这两句话的词性标记相同，但分块方式不同。在第一句中，the farmer 和 rice 都是单独
的块，而在第二个句子中相应的部分，the computer monitor，是一个单独的块。显然，如
果我们想最大限度地提升分块的性能，我们需要使用词的内容信息作为词性标记的补充。
"""

'\n有时词性标记不足以确定一个句子应如何分块。例如：考虑下面的两个语句：\n\na. Joey/NN sold/VBD the/DT farmer/NN rice/NN ./.\nb. Nick/NN broke/VBD my/DT computer/NN monitor/NN ./.\n\n这两句话的词性标记相同，但分块方式不同。在第一句中，the farmer 和 rice 都是单独\n的块，而在第二个句子中相应的部分，the computer monitor，是一个单独的块。显然，如\n果我们想最大限度地提升分块的性能，我们需要使用词的内容信息作为词性标记的补充。\n'

In [ ]:
# 级联分块器构建嵌套结构
